In [ ]:
# Estos dos comandos evitan que haya que hacer reload cada vez que se modifica un paquete
%load_ext autoreload
%autoreload 2

# Ejercicio de clasificación de texto

Naive Bayes es una técnica estadística que consiste en repetir el método anterior en problemas cuyos sucesos no son independientes, pero suponiendo independencia.
A lo largo de este trabajo desarrollarán un modelo de Naive Bayes para el problema de clasificación de artículos periodístios.En este caso podemos estimar la probabilidad de ocurrencia de cada palabra según la categoría a la que pertenece el artículo.

## Dataset


El primer paso es obtener el dataset que vamos a utilizar. El dataset a utilizar es el de TwentyNewsGroup(TNG) que está disponible en sklearn.

Se puede encontrar más información del dataset en la documentación de scikit-learn.

In [7]:
#Loading the data set - training data.
from sklearn.datasets import fetch_20newsgroups
from os.path import isfile
import pickle

TT_FILE = 'twenty-train.p'
if isfile(TT_FILE):
    twenty_train = pickle.load(open(TT_FILE, 'rb'))
else:
    twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
    pickle.dump(twenty_train, open(TT_FILE, 'wb'))

El siguiente paso es analizar el contenido del dataset, como por ejemplo la cantidad de artículos, la cantidad de clases, etc.

Preguntas:

1) ¿Cuántos articulos tiene el dataset?

In [18]:
len(twenty_train.data)

11314

2) ¿Cuántas clases tiene el dataset?

In [19]:
len(twenty_train.target_names)

20

3) ¿Es un dataset balanceado?

In [39]:
import numpy as np
_, counts = np.unique(twenty_train.target, return_counts=True)
if len(set(counts)) == 1:
    print('El dataset está balanceado')
else:    
    print('El dataset no está balanceado')

El dataset no está balanceado


4) ¿Cuál es la probabilidad a priori de la clase 5? A que corresponde esta clase?

In [41]:
priori5 = counts[5]/sum(counts)
print(f'La clase 5 ({twenty_train.target_names[5]}) tiene probabilidad a priori {priori5:.2}')

La clase 5 (comp.windows.x) tiene probabilidad a priori 0.052


5) ¿Cuál es la clase con mayor probabilidad a priori?

In [45]:
max_class = np.argmax(counts)
print(f'La clase {max_class} ({twenty_train.target_names[max_class]}) tiene la máxima probabilidad a priori, {counts[max_class]/sum(counts):.2}')

La clase 10 (rec.sport.hockey) tiene la máxima probabilidad a priori, 0.053


## Preprocesamiento

Para facilitar la comprensión de los algoritmos de preprocesamiento, se aplican primero a un solo artículo.


Mas info en:
http://text-processing.com/demo/stem/

In [47]:
article = twenty_train.data[0]
article

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

- **Tokenization (nltk):**

In [48]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

tok = word_tokenize(article)
tok

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rochi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


['From',
 ':',
 'lerxst',
 '@',
 'wam.umd.edu',
 '(',
 'where',
 "'s",
 'my',
 'thing',
 ')',
 'Subject',
 ':',
 'WHAT',
 'car',
 'is',
 'this',
 '!',
 '?',
 'Nntp-Posting-Host',
 ':',
 'rac3.wam.umd.edu',
 'Organization',
 ':',
 'University',
 'of',
 'Maryland',
 ',',
 'College',
 'Park',
 'Lines',
 ':',
 '15',
 'I',
 'was',
 'wondering',
 'if',
 'anyone',
 'out',
 'there',
 'could',
 'enlighten',
 'me',
 'on',
 'this',
 'car',
 'I',
 'saw',
 'the',
 'other',
 'day',
 '.',
 'It',
 'was',
 'a',
 '2-door',
 'sports',
 'car',
 ',',
 'looked',
 'to',
 'be',
 'from',
 'the',
 'late',
 '60s/',
 'early',
 '70s',
 '.',
 'It',
 'was',
 'called',
 'a',
 'Bricklin',
 '.',
 'The',
 'doors',
 'were',
 'really',
 'small',
 '.',
 'In',
 'addition',
 ',',
 'the',
 'front',
 'bumper',
 'was',
 'separate',
 'from',
 'the',
 'rest',
 'of',
 'the',
 'body',
 '.',
 'This',
 'is',
 'all',
 'I',
 'know',
 '.',
 'If',
 'anyone',
 'can',
 'tellme',
 'a',
 'model',
 'name',
 ',',
 'engine',
 'specs',
 ',',
 'y


- **Lemmatization (nltk):**


In [50]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
lem=[lemmatizer.lemmatize(x,pos='v') for x in tok]
lem

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rochi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['From',
 ':',
 'lerxst',
 '@',
 'wam.umd.edu',
 '(',
 'where',
 "'s",
 'my',
 'thing',
 ')',
 'Subject',
 ':',
 'WHAT',
 'car',
 'be',
 'this',
 '!',
 '?',
 'Nntp-Posting-Host',
 ':',
 'rac3.wam.umd.edu',
 'Organization',
 ':',
 'University',
 'of',
 'Maryland',
 ',',
 'College',
 'Park',
 'Lines',
 ':',
 '15',
 'I',
 'be',
 'wonder',
 'if',
 'anyone',
 'out',
 'there',
 'could',
 'enlighten',
 'me',
 'on',
 'this',
 'car',
 'I',
 'saw',
 'the',
 'other',
 'day',
 '.',
 'It',
 'be',
 'a',
 '2-door',
 'sport',
 'car',
 ',',
 'look',
 'to',
 'be',
 'from',
 'the',
 'late',
 '60s/',
 'early',
 '70s',
 '.',
 'It',
 'be',
 'call',
 'a',
 'Bricklin',
 '.',
 'The',
 'doors',
 'be',
 'really',
 'small',
 '.',
 'In',
 'addition',
 ',',
 'the',
 'front',
 'bumper',
 'be',
 'separate',
 'from',
 'the',
 'rest',
 'of',
 'the',
 'body',
 '.',
 'This',
 'be',
 'all',
 'I',
 'know',
 '.',
 'If',
 'anyone',
 'can',
 'tellme',
 'a',
 'model',
 'name',
 ',',
 'engine',
 'specs',
 ',',
 'years',
 'of',


- **Stop Words (nltk):**


In [51]:
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = [x for x in lem if x not in stopwords.words('english')]
stop

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rochi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


['From',
 ':',
 'lerxst',
 '@',
 'wam.umd.edu',
 '(',
 "'s",
 'thing',
 ')',
 'Subject',
 ':',
 'WHAT',
 'car',
 '!',
 '?',
 'Nntp-Posting-Host',
 ':',
 'rac3.wam.umd.edu',
 'Organization',
 ':',
 'University',
 'Maryland',
 ',',
 'College',
 'Park',
 'Lines',
 ':',
 '15',
 'I',
 'wonder',
 'anyone',
 'could',
 'enlighten',
 'car',
 'I',
 'saw',
 'day',
 '.',
 'It',
 '2-door',
 'sport',
 'car',
 ',',
 'look',
 'late',
 '60s/',
 'early',
 '70s',
 '.',
 'It',
 'call',
 'Bricklin',
 '.',
 'The',
 'doors',
 'really',
 'small',
 '.',
 'In',
 'addition',
 ',',
 'front',
 'bumper',
 'separate',
 'rest',
 'body',
 '.',
 'This',
 'I',
 'know',
 '.',
 'If',
 'anyone',
 'tellme',
 'model',
 'name',
 ',',
 'engine',
 'specs',
 ',',
 'years',
 'production',
 ',',
 'car',
 'make',
 ',',
 'history',
 ',',
 'whatever',
 'info',
 'funky',
 'look',
 'car',
 ',',
 'please',
 'e-mail',
 '.',
 'Thanks',
 ',',
 '-',
 'IL',
 '--',
 '--',
 'bring',
 'neighborhood',
 'Lerxst',
 '--',
 '--']

- **Stemming (nltk):**


In [52]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stem=[stemmer.stem(x) for x in stop]
stem

['from',
 ':',
 'lerxst',
 '@',
 'wam.umd.edu',
 '(',
 "'s",
 'thing',
 ')',
 'subject',
 ':',
 'what',
 'car',
 '!',
 '?',
 'nntp-posting-host',
 ':',
 'rac3.wam.umd.edu',
 'organ',
 ':',
 'univers',
 'maryland',
 ',',
 'colleg',
 'park',
 'line',
 ':',
 '15',
 'I',
 'wonder',
 'anyon',
 'could',
 'enlighten',
 'car',
 'I',
 'saw',
 'day',
 '.',
 'It',
 '2-door',
 'sport',
 'car',
 ',',
 'look',
 'late',
 '60s/',
 'earli',
 '70',
 '.',
 'It',
 'call',
 'bricklin',
 '.',
 'the',
 'door',
 'realli',
 'small',
 '.',
 'In',
 'addit',
 ',',
 'front',
 'bumper',
 'separ',
 'rest',
 'bodi',
 '.',
 'thi',
 'I',
 'know',
 '.',
 'If',
 'anyon',
 'tellm',
 'model',
 'name',
 ',',
 'engin',
 'spec',
 ',',
 'year',
 'product',
 ',',
 'car',
 'make',
 ',',
 'histori',
 ',',
 'whatev',
 'info',
 'funki',
 'look',
 'car',
 ',',
 'pleas',
 'e-mail',
 '.',
 'thank',
 ',',
 '-',
 'IL',
 '--',
 '--',
 'bring',
 'neighborhood',
 'lerxst',
 '--',
 '--']

- **Filtrado de palabras:**


In [53]:
alpha = [x for x in stem if x.isalpha()]
alpha

['from',
 'lerxst',
 'thing',
 'subject',
 'what',
 'car',
 'organ',
 'univers',
 'maryland',
 'colleg',
 'park',
 'line',
 'I',
 'wonder',
 'anyon',
 'could',
 'enlighten',
 'car',
 'I',
 'saw',
 'day',
 'It',
 'sport',
 'car',
 'look',
 'late',
 'earli',
 'It',
 'call',
 'bricklin',
 'the',
 'door',
 'realli',
 'small',
 'In',
 'addit',
 'front',
 'bumper',
 'separ',
 'rest',
 'bodi',
 'thi',
 'I',
 'know',
 'If',
 'anyon',
 'tellm',
 'model',
 'name',
 'engin',
 'spec',
 'year',
 'product',
 'car',
 'make',
 'histori',
 'whatev',
 'info',
 'funki',
 'look',
 'car',
 'pleas',
 'thank',
 'IL',
 'bring',
 'neighborhood',
 'lerxst']

### Preprocesamiento completo

Utilizar o no cada uno de los métodos vistos es una decisión que dependerá del caso particular de aplicación. Para este ejercicio vamos a considerar las siguientes combinaciones:

- Tokenización
- Tokenización, Lematización, Stemming.
- Tokenización, Stop Words.
- Tokenización, Lematización, Stop Words, Stemming.
- Tokenización, Lematización, Stop Words, Stemming, Filtrado.

In [ ]:
#Solución

Preguntas

- Cómo cambia el tamaño del vocabulario al agregar Lematización y Stemming?
- Cómo cambia el tamaño del vocabulario al Stop Words?
- Analice muy brevemente ventajas y desventajas del tamaño del dataset en cada caso.

### Guardado de pre procesamiento
Vamos a guardar lo preprocesado usando pickle, que nos permite serializar objetos y guardarlos en disco, es muy importante que sepan hacer esto si no quieren perder tiempo!

In [ ]:
#Salvado del procesamiento a disco:
import pickle

with open('art_filt.pck', 'wb') as fp:
    pickle.dump(emails_filtrados, fp)

In [ ]:
with open ('art_filt.pck', 'rb') as fp:
    itemlist = pickle.load(fp)

## Vectorización de texto

- **Obtención del vocabulario y obtención de la probabilidad**

Como se vió en clase, los vectorizadores cuentan con dos parámetros de ajuste.

- max_df: le asignamos una maxima frecuencia de aparición, eliminando las palabras comunes que no aportan información.

- min_df: le asignamos la minima cantidad de veces que tiene que aparecer una palabra.


Al igual que con las diferentes opciones de preprocesamiento, lo mismo ocurre con la vectorización. Podemos utilizar CountVectorizer o TfidfVectorizer según el caso (con diferentes valores de max_df y min_df).
Para este ejercicio deben utilizar ambos métodos.

In [ ]:
#Solución

In [ ]:
raw_data.shape #Para cada documento hay un vector de ocurrencias

In [ ]:
raw_data.toarray() #Es una sparse matrix, vamos a expandirla

In [ ]:
raw_data.toarray()[0,:].argmax() #Veamos a qué palabra pertenece la máxima ocurrencia en el primer artíclo

## Entrenamiento del modelo

Primero deben separar correctamente el dataset para hacer validación del modelo.

Y luego deben entrenar el modelo de NaiveBayes con el dataset de train.

Deben utilizar un modelo de NaiveBayes Multinomial y de Bernoulli. Ambos modelos estan disponibles en sklearn.

In [ ]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

#Completar para cada caso según corresponda
clf = MultinomialNB()
# clf = BernoulliNB()
clf.fit(X_train_data.toarray(), Y_train_data)


Finalmente comprobar el accuracy en train.

In [ ]:
import numpy as np
porc=sum(np.array(clf.predict(X_val_train.toarray()))==np.array(Y_train_data))/len(Y_train_data)*100
print("El porcentaje de artículos clasificados correctamente es: {:.2f}%".format(porc))

Preguntas

- Con que combinación de preprocesamiento obtuvo los mejores resultados? Explique por qué cree que fue así.

- Con que modelo obtuvo los mejores resultados? Explique por qué cree que fue así.

## Performance de los modelos

En el caso anterior, para medir la cantidad de artículos clasiicados correctamente se utilizó el mismo subconjunto del dataset que se utilizó para entrenar.

Esta medida no es una medida del todo útil, ya que lo que interesa de un clasificador es su capacidad de clasificación de datos que no fueron utilizados para entrenar. Es por eso que se pide, para el clasificador entrenado con el subconjunto de training, cual es el porcentaje de artículos del subconjunto de testing clasificados correctamente. Comparar con el porcentaje anterior y explicar las diferencias.

Finalmente deben observar las diferencias y extraer conclusiones en base al accuracy obtenido, el preprocesamiento y vectorización utilizado y el modelo, para cada combinación de posibilidades.

In [ ]:
# Solución

In [ ]:
import numpy as np
porc=sum(np.array(clf.predict(X_test.toarray()))==np.array(Y_test))/len(Y_test)*100
print("El porcentaje de artículos clasificados correctamente es: {:.2f}%".format(porc))

Preguntas

- El accuracy en el dataset de test es mayor o menor que en train? Explique por qué.